In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
import gym
import numpy as np

# Define Policy Network
class Policy(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Policy, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, output_dim)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.softmax(self.fc3(x))
        return x

def discount_rewards(rewards, gamma=0.99):
    discounted_rewards = []
    running_add = 0
    for t, r in enumerate(reversed(rewards)):
        # print(t)
        running_add = running_add * gamma ** t + r
        discounted_rewards.insert(0, running_add)
    return discounted_rewards

# Initialize the environment
env = gym.make('CartPole-v1')
input_dim = env.observation_space.shape[0]
output_dim = env.action_space.n

# Initialize the policy network
policy = Policy(input_dim, output_dim)

# Initialize the optimizer
optimizer = optim.Adam(policy.parameters(), lr=0.01)

# Training loop
for episode in range(1):
    state = env.reset()
    episode_reward = 0

    states = []
    actions = []
    rewards = []

    while True:
        env.render()  # Render the environment

        states.append(torch.tensor(state, dtype=torch.float32))

        action_probs = policy(torch.tensor(state, dtype=torch.float32))

        # take the action based on the probability. Higher probability will have a higher chance, but not always
        action = torch.multinomial(action_probs, num_samples=1).item() 
        
        actions.append(action)

        # print("Action Probs: {}, Action: {}".format(action_probs, action))
        next_state, reward, done, _ = env.step(action)
        rewards.append(reward)

        episode_reward += reward
        state = next_state

        if done:
            # Compute discounted rewards
            discounted_rewards = torch.tensor(discount_rewards(rewards), dtype=torch.float32)

            # Compute policy loss
            log_probs = torch.log(policy(torch.stack(states)))[range(len(actions)), actions]
            policy_loss = -(log_probs * discounted_rewards).mean()

            # print("States: {}, Actions: {}".format(policy(torch.stack(states)), [range(len(actions)), actions]))

            # Update policy
            optimizer.zero_grad()
            policy_loss.backward()
            optimizer.step()

            print("Episode: {}, Reward: {}".format(episode, episode_reward))
            break

env.close()


rewards: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
discounted_rewards: tensor([3.6499, 3.7671, 3.8943, 4.0325, 4.1829, 4.3464, 4.5240, 4.7165, 4.9244,
        5.1478, 5.3864, 5.6393, 5.9049, 6.1804, 6.4624, 6.7459, 7.0252, 7.2929,
        7.5408, 7.7595, 7.9387, 8.0677, 8.1356, 8.1315, 8.0456, 7.8692, 7.5955,
        7.2199, 6.7406, 6.1590, 5.4797, 4.7106, 3.8628, 2.9504, 1.9900, 1.0000])
Episode: 0, Reward: 36.0
